* Twitter data will be pre processed as numerical data for the ML models to
understand.
* The dataset is split into train and test dataset.
* Logistic regression model
* Identify whether Positive/ negative tweet

Dataset: https://www.kaggle.com/datasets/kazanova/sentiment140

In [ ]:
! pip install kaggle

In [2]:
# Configuring the path of Kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing Twitter sentiment dataset

In [3]:
import kagglehub
import shutil

# Download the dataset
path = kagglehub.dataset_download("kazanova/sentiment140")

# Define the destination directory in the content folder
destination_dir = "/content/sentiment140"

# Move the dataset to the content folder
shutil.move(path, destination_dir)

print("Path to dataset files in content folder:", destination_dir)

100%|██████████| 80.9M/80.9M [00:01<00:00, 59.1MB/s]

Extracting files...


Path to dataset files in content folder: /content/sentiment140


In [ ]:
# extract the compressed zip file

# from zipfile import ZipFile

# with ZipFile(path, 'r') as zip:
#   zip.extractall()
#   print("Done")

In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Data Processing**

In [7]:
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']

In [8]:
# loading the data from csv file to pandas df

df = pd.read_csv('/content/sentiment140/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')

In [9]:
twitter_data = df.copy()

In [10]:
twitter_data.shape

(1600000, 6)

In [11]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
# Counting for missing values
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [13]:
# checking the distribution of the target variable
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


if the distribution is not even have to perform upsampling / downsampling. but here the dataset is even of positive / negative

In [14]:
# Convert 4 to 1
twitter_data.replace({'target':{4:1}}, inplace=True)

In [15]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


* 0 --->Negative tweet
* 1 --->Positive tweet

**Stemming**
* Stemming is the part of reducing to its root word
* example: actors, actress, acting ---> act

In [16]:
port_stem = PorterStemmer()

In [ ]:
# def stemming(content):
#   stemmed_content = re.sub('[^a-zA-Z]',' ',content)
#   stemmed_content = stemmed_content.lower()
#   stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
#   stemmed_content = ' '.join(stemmed_content)
#   return stemmed_content

In [ ]:
# twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [17]:
# Initialize stemmer and stopwords
stop_words = set(stopwords.words('english'))

def stemming(content):
    # Step 1: Remove non-alphabetic characters and convert to lowercase
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()

    # Step 2: Tokenize into words
    words = stemmed_content.split()

    # Step 3: Remove stopwords and apply stemming
    stemmed_words = [port_stem.stem(word) for word in words if word not in stop_words]

    # Step 4: Join the stemmed words back into a single string
    stemmed_content = ' '.join(stemmed_words)

    return stemmed_content

In [19]:
from tqdm import tqdm
tqdm.pandas()

In [20]:
twitter_data['stemmed_content'] = twitter_data['text'].progress_apply(stemming)

100%|██████████| 1600000/1600000 [04:30<00:00, 5905.79it/s]


In [21]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [22]:
twitter_data.to_csv('twitter.csv', index=False)

In [23]:
df_loaded = pd.read_csv('twitter.csv')

# Continue working
print(df_loaded)

         target         ids                          date      flag  \
0             0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1             0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2             0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3             0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4             0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
...         ...         ...                           ...       ...   
1599995       1  2193601966  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599996       1  2193601969  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599997       1  2193601991  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599998       1  2193602064  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599999       1  2193602129  Tue Jun 16 08:40:50 PDT 2009  NO_QUERY   

                    user                                               text  \
0        _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Aww

In [24]:
# seperating the data and the label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values


In [25]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


Splitting the data into train and test

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

# equal propotion of negeative and positive tweets in train and test dataset

In [27]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [28]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [29]:
print(X_train)

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

Training the ML model

In [30]:
model = LogisticRegression(max_iter=1000)

In [31]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

In [32]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [33]:
print('Accuracy on training data : ', training_data_accuracy)

Accuracy on training data :  0.79871953125


In [34]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [35]:
print('Accuracy on test data : ', test_data_accuracy)

Accuracy on test data :  0.77668125


Model accuracy - 77.6%

In [36]:
# Saving the trained model
import pickle
file_name = 'trained_model.sav'
pickle.dump(model, open(file_name, 'wb'))

In [38]:
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [39]:
X_new = X_test[200]
print(Y_test[200])

1


In [40]:
prediction_m = model.predict(X_new)
print(prediction_m)

if (prediction_m[0]==0):
  print('The tweet is Negative')
else:
  print('The tweet is Positive')

[1]
The tweet is Positive


double check with the loaded model

In [41]:
prediction_lm = loaded_model.predict(X_new)
print(prediction_lm)

if (prediction_lm[0]==0):
  print('The tweet is Negative')
else:
  print('The tweet is Positive')

[1]
The tweet is Positive


In [42]:
X_new = X_test[3]
print(Y_test[3])
prediction_m = model.predict(X_new)
print(prediction_m)

if (prediction_m[0]==0):
  print('The tweet is Negative')
else:
  print('The tweet is Positive')

0
[0]
The tweet is Negative
